In [1]:
import tensorflow as tf

In [2]:
from pitycnn.inputs import Inputs
from pitycnn.prep import prepare_data
import numpy as np

In [3]:
batch_size = 32
image_width = 234
image_height = 234
image_depth = 3
num_labels = 120
learning_rate = 0.001
num_epochs = 10
display_step = 1
filewriter_path = "model"

In [4]:
def block(inputs, filters, name, dropout, is_training):

    conv1 = tf.layers.conv2d(inputs,
                             filters,
                             4,
                             padding="same",
                             activation=tf.nn.relu,
                             name="conv{}_1".format(name))
    conv2 = tf.layers.conv2d(conv1,
                             filters,
                             4,
                             padding="same",
                             activation=tf.nn.relu,
                             name="conv{}_2".format(name))
    conv3 = tf.layers.conv2d(conv2,
                             filters,
                             4,
                             padding="same",
                             activation=tf.nn.relu,
                             name="conv{}_3".format(name))
    pool = tf.layers.max_pooling2d(conv3,
                                   pool_size=2,
                                   strides=2,
                                   name="pool{}".format(name))

    dropout = tf.layers.dropout(pool, dropout, training=is_training, name="dropout{}".format(name))
    return dropout

In [5]:
def model(inputs, is_training, output_size):
    filters_size = [64, 128, 256, 512, 512]
    filter_names = ['64', '128', '256', '512a', '512b']
    dropouts = [0.1, 0.2, 0.3, 0.5, 0.5]
    
    with tf.name_scope("covnet"):
        inputs = tf.layers.batch_normalization(inputs, name="batch_normalization")

        for i, filter_size in enumerate(filters_size):
            with tf.name_scope("conv-maxpool-{}s".format(filter_names[i])):
                inputs = block(inputs, filter_size, filter_names[i], dropouts[i], is_training)

        with tf.name_scope("conv-dense"):
            inputs = tf.reshape(inputs, [-1, 7 * 7 * 512])
            dense1 = tf.layers.dense(
                inputs,
                4096,
                activation=tf.nn.relu,
                name='dense1',
            )
            dropout = tf.layers.dropout(dense1, 0.5, training=is_training, name="dense_dropout")
            dense2 = tf.layers.dense(
                dropout,
                4096,
                activation=tf.nn.relu,
                name='dense2',
            )
            net = tf.layers.dense(
                dense2,
                output_size,
                name='dense3',
            )
        return net
        

In [6]:
#1) First we put the input data in a tensorflow friendly form. 
tf_train_dataset = tf.placeholder(tf.float32, shape=(None, image_width, image_height, image_depth), name="inputs")
tf_train_labels = tf.placeholder(tf.float32, shape = (None, num_labels), name="labels")

logits = model(tf_train_dataset, True, num_labels)

with tf.name_scope("loss"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels))

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

with tf.name_scope("predictions"):
    train_prediction = tf.nn.softmax(logits)
    
tf.summary.scalar('cross_entropy', loss)

with tf.name_scope("accuracy"):
        correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(tf_train_labels, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
       
tf.summary.scalar('accuracy', accuracy)
merged_summary = tf.summary.merge_all()
writer = tf.summary.FileWriter(filewriter_path)
saver = tf.train.Saver()


In [9]:
files = '/Users/fulop/Downloads/dogbreed/sample/';

train_images, train_labels, valid_images, valid_labels, n_classes = prepare_data(files)

train_inputs = Inputs(train_images, train_labels, n_classes, batch_size=batch_size, shuffle=True)
valid_inputs = Inputs(valid_images, valid_labels, n_classes, name="valid", batch_size=batch_size)

with tf.device('/cpu:0'):
    iterator_train = train_inputs.generate_iterator()
    iterator_valid = valid_inputs.generate_iterator()
    next_batch = iterator_train.get_next()
    next_batch_valid = iterator_valid.get_next()

train_batches_per_epoch = int(np.floor(len(train_inputs.labels)/batch_size))

valid_batches_per_epoch = int(np.floor(len(valid_inputs.labels)/batch_size))


with tf.Session() as sess:
     
    tf.global_variables_initializer().run()
    writer.add_graph(sess.graph)

    for epoch in range(num_epochs):

        sess.run(iterator_train.initializer)


        for step in range(1):
            batch_data, batch_labels = sess.run(next_batch)
            feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
            _, l, predictions = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

            if step % display_step == 0:
                s = sess.run(merged_summary, feed_dict={tf_train_labels: batch_labels,
                                                        logits: predictions})

                writer.add_summary(s, epoch*train_batches_per_epoch + step)
        
        sess.run(iterator_valid.initializer)
        valid_acc = 0.
        valid_count = 0
        for _ in range(valid_batches_per_epoch):

            batch_data, batch_labels = sess.run(next_batch_valid)
            acc = sess.run(accuracy, feed_dict={tf_train_labels: batch_labels,
                                                tf_train_dataset: batch_data})#fix dropout
            test_acc += acc
            test_count += 1
        test_acc /= test_count
        print("{} Validation Accuracy = {:.4f}".format(datetime.now(),
                                                       test_acc))
        
        checkpoint_name = os.path.join(filewriter_path,
                                       'model_epoch'+str(epoch+1)+'.ckpt')
        save_path = saver.save(sess, checkpoint_name)


NameError: name 'test_acc' is not defined

In [ ]:
print("lol")